In [2]:
import os

from dotenv import load_dotenv
load_dotenv()

DB_NAME = os.environ.get("DB_NAME")
DB_COLLECTION = os.environ.get("DB_COLLECTION")

from pymongo import MongoClient
client = MongoClient('localhost:27017')
db = client[DB_NAME]
coll = db[DB_COLLECTION]

print(coll.count_documents({}))

TypeError: name must be an instance of str

In [2]:
import pandas as pd
df = pd.DataFrame(list(coll.find()))

In [3]:
df = df.drop(['english_headline', 'date_accessed_UTC', 'date_updated_UTC', 's3URL', 'post_category', 'claims_review'], axis = 1)

In [4]:
df['date'] =  pd.to_datetime(df['date_accessed'])

In [5]:
filtered_df = df.loc[(df['date'] >= '2020-09-19')& (df['date'] < '2020-09-26')]

In [6]:
filtered_df['factcheck_site'] = filtered_df['domain']

C:\Users\user\anaconda3\lib\site-packages\pandas\core\frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)


In [7]:
filtered_df.factcheck_site = filtered_df.factcheck_site.apply(lambda x: 'boomlive' if 'boomlive' in x else x)
filtered_df.factcheck_site = filtered_df.factcheck_site.apply(lambda x: 'altnews' if 'altnews' in x else x)
filtered_df.factcheck_site = filtered_df.factcheck_site.apply(lambda x: 'factly' if 'factly' in x else x)
filtered_df.factcheck_site = filtered_df.factcheck_site.apply(lambda x: 'newsmobile' if 'newsmobile' in x else x)
filtered_df.factcheck_site = filtered_df.factcheck_site.apply(lambda x: 'indiatoday' if 'indiatoday' in x else x)
filtered_df['factcheck_site'].value_counts()

boomlive      40
newsmobile    30
altnews       22
factly        19
indiatoday     9
Name: factcheck_site, dtype: int64

In [8]:
filtered_df.drop_duplicates(subset = "postURL", inplace = True)

C:\Users\user\anaconda3\lib\site-packages\pandas\util\_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


In [17]:
filtered_df.to_csv(r'C:\Users\user\Desktop\factcheck sites database .csv', index=False)

Summary Statistics

In [18]:
# Count of articles
len(pd.unique(filtered_df['postURL']))

120

In [19]:
# Daywise count of articles
df = filtered_df.groupby(['date']).size().reset_index(name='counts')

,date,counts
0,2020-09-21,120


In [ ]:
fig = px.line(df, x="date", y="counts")
fig.show()

In [20]:
# domain wise count of articles
filtered_df.groupby(['factcheck_site']).size().reset_index(name='counts')

,factcheck_site,counts
0,altnews,22
1,boomlive,40
2,factly,19
3,indiatoday,9
4,newsmobile,30


In [9]:
filtered_df = filtered_df.explode('docs').reset_index(drop=True)
filtered_df['media_type'] = filtered_df['docs'].apply(lambda x: x.get('mediaType'))

In [22]:
# count of media types and social media posts -
filtered_df['media_type'].value_counts()

image    171
text     120
tweet     49
video     33
Name: media_type, dtype: int64

In [23]:
# daywise count of media types and social media posts
filtered_df.groupby('date')['media_type'].value_counts()

date        media_type
2020-09-21  image         171
            text          120
            tweet          49
            video          33
Name: media_type, dtype: int64

In [24]:
#domain wise count of media types and social media posts
filtered_df.groupby('factcheck_site')['media_type'].value_counts()

factcheck_site  media_type
altnews         image         60
                text          22
                video          8
boomlive        image         53
                tweet         41
                text          40
                video          3
factly          image         41
                text          19
                tweet          5
indiatoday      image         17
                text           9
                video          4
                tweet          3
newsmobile      text          30
                video         18
Name: media_type, dtype: int64

In [29]:
#Number and percentage of articles with tweets/fb posts
a = len(pd.unique(filtered_df[filtered_df['media_type']=="tweet"]['postURL']))
b = len(pd.unique(filtered_df[filtered_df['media_type']=="fbpost"]['postURL']))
c = len(pd.unique(filtered_df['postURL']))
print("number of articles with tweets:",a)
print("percentage of articles with tweets:",(a/c)*100,"%")
print("number of articles with fb posts:",b)
print("percentage of articles with fb posts:",(b/c)*100,"%")

number of articles with tweets: 32
percentage of articles with tweets: 26.666666666666668 %
number of articles with fb posts: 0
percentage of articles with fb posts: 0.0 %


In [33]:
#Average number of images per article 
a = filtered_df[filtered_df['media_type']=="image"].groupby(['postURL']).size().reset_index(name='counts')['counts'].mean()
print(round(a))

2
